AI4Code Pytorch DistilBert Baseline¶
I used a lot of code from Kaggle's starter notebook here: https://www.kaggle.com/code/ryanholbrook/getting-started-with-ai4code
this notebook explin how to extract functions,classes,variables....names ,so we can use them in a clean data, and improve score

# Setup #

In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('../input/AI4Code')

# Load Data #

The notebooks are stored as individiual JSON files. They've been cleaned of the usual metadata present in Jupyter notebooks, leaving only the `cell_type` and `source`. The [Data](https://www.kaggle.com/competitions/AI4Code/data) page on the competition website has the full documentation of this dataset.

We'll load the notebooks here and join them into a dataframe for easier processing. The full set of training data takes quite a while to load, so we'll just use a subset for this demonstration.

In [ ]:
NUM_TRAIN = 200


def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
        .assign(id=path.stem)
        .rename_axis('cell_id')
    )


paths_train = list((data_dir / 'train').glob('*.json'))[:NUM_TRAIN]
notebooks_train = [
    read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
]
df = (
    pd.concat(notebooks_train)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)

df

Each notebook has all the code cells given first with the markdown cells following. The code cells are in the correct relative order, while the markdown cells are shuffled. In the next section, we'll see how to recover the correct orderings for notebooks in the training set.

In [ ]:
# Get an example notebook
nb_id = df.index.unique('id')[6]
print('Notebook:', nb_id)

print("The disordered notebook:")
nb = df.loc[nb_id, :]
display(nb)
print()

Your task in this competition is to predict the correct order of the notebook cells, both code and markdown. Since you're given the relative ordering of the code cells among themselves, you could also think of this as predicting where the markdown cells should be placed among the code cells.

For example, a disordered notebook might be:
```
code_1
code_2
code_3
markdown_1
markdown_2
```
and the correctly ordered notebook might be:
```
code_1
markdown_2
code_2
code_3
markdown_1
```
The markdown cells can be in any order, but you would never see `code_2` before `code_1`, for instance.

# Ordering the Cells #

In the `train_orders.csv` file we have, for notebooks in the training set, the correct ordering of cells in terms of the cell ids.

In [ ]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()  # Split the string representation of cell_ids into a list

df_orders

In [ ]:
# Get the correct order
cell_order = df_orders.loc[nb_id]

print("The ordered notebook:")
nb.loc[cell_order, :]

The correct numeric position of a cell we will call the **rank** of the cell. We can find the ranks of the cells within a notebook by referencing the true ordering of cell ids as given in `train_orders.csv`.

In [ ]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

cell_ranks = get_ranks(cell_order, list(nb.index))
nb.insert(0, 'rank', cell_ranks)

nb

In [ ]:
code=nb.loc[nb.cell_type=="code"]

In [ ]:
code

In [ ]:
import tokenize
import io

In [ ]:
code.loc['d3e265f6','source']

In [ ]:
code_text = tokenize.generate_tokens(io.StringIO(code.loc['d3e265f6','source']).readline)
[tok for tok in code_text]

In [ ]:
#extract only function names , variables,comments then we can join them
code_text = tokenize.generate_tokens(io.StringIO(code.loc['d3e265f6','source']).readline)
[tok.string for tok in code_text if tok.type==1 or tok.type==55]


In [ ]:
#join all strings
code_text = tokenize.generate_tokens(io.StringIO(code.loc['d3e265f6','source']).readline)
' '.join([tok.string for tok in code_text if tok.type==1 or tok.type==55])